# 07-Das Grundprinzip von Gruppieren
Der Prozess des Gruppieren ("group by") bezieht sich auf ein oder mehrere Handlung:
* **Splitting** Also das Zerlegen des Datensatzes in bestimmte Gruppen
* **Applying** Anwenden einer Funktion auf jede dieser Gruppen
* **Combining** Zusammenfassen der Daten in eine neue Datenstruktur

Das **Splitting** ist der zentrale Schritt und wird oft als erstes ausgeführt. Meisten sollen die Datensätze in den Gruppen aber noch weiterverarbeitet werden. Daher stehenim Schritt **Applying** unterem anderem die folgenden Funktionen zur Verfügung:

* **Aggregation:** Kummulieren der Daten mit Statistischen Methoden, wie zum Beispiel:
    * Summen und Mittelwerte
    * Zählen der Werte
    
    
* **Transformation:**
    * Standardisierung der Daten innerhalb einer Gruppe
    * Aufüllen von fehlenden Daten innerhalb einrer Gruppe mit aus der Gruppe abgeleitetend Werte

* **Filtration:**
    * Gruppenbezogenes Filtern, z.B. das Aussortieren von Ausreißern innerhalb einer Gruppe
    

* Oder aber die Kombination verschiedener Schritte


---
*Übersetz aus folgender Quelle: https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html*



# Arbeiten mit ``.groupby()``
## Datageneration

In [23]:
import pandas as pd
import numpy as np

In [24]:
# Generieren von Übungsdaten
df_students = pd.DataFrame({'student_id':list(range(100,107)), 
                            'subject_id':[1,2,3,1,2,3,1],
                            'subject': ['Math', 'Geo', 'Business', 'Math', 'Geo', 'Business' ,'Math'],
                            'age':[18,22,31,22,26,21,20],
                            'left_hand':[False,True,False,True,False,False,True]})
df_students

,student_id,subject_id,subject,age,left_hand
0,100,1,Math,18,False
1,101,2,Geo,22,True
2,102,3,Business,31,False
3,103,1,Math,22,True
4,104,2,Geo,26,False
5,105,3,Business,21,False
6,106,1,Math,20,True


## Zählen von Datensätzen ``.groupby().count(<column>)``

In [25]:
# Mit groupby.count() können wir uns anzeigen lassen, wie viel Datensätze jede Gruppe besitzt 
df_students.groupby('subject').count()

,student_id,subject_id,age,left_hand
subject,,,,
Business,2,2,2,2
Geo,2,2,2,2
Math,3,3,3,3


In [26]:
# Alternativ können wir auch nach der Größe fragen
df_students.groupby('subject').size()

subject
Business    2
Geo         2
Math        3
dtype: int64

**Unterschied zwischen ``.count()`` und ``.size()``**

In [27]:
# Unterschied zwischen .count() und .size()
df_students_nan = pd.DataFrame({'student_id':list(range(100,107)), 
                            'subject_id':[1,2,3,1,2,np.NaN,1],
                            'subject': ['Math', 'Geo', 'Business', 'Math', 'Geo', 'Business' ,'Math'],
                            'age':[np.NaN,22,31,22,26,21,20],
                           'left_hand':[False,True,False,True,False,False,True]})
df_students_nan

,student_id,subject_id,subject,age,left_hand
0,100,1.0,Math,NaN,False
1,101,2.0,Geo,22.0,True
2,102,3.0,Business,31.0,False
3,103,1.0,Math,22.0,True
4,104,2.0,Geo,26.0,False
5,105,NaN,Business,21.0,False
6,106,1.0,Math,20.0,True


In [28]:
# count wird durch NaN Werte beeinflusst
df_students_nan.groupby('subject').count()

,student_id,subject_id,age,left_hand
subject,,,,
Business,2,1,2,2
Geo,2,2,2,2
Math,3,3,2,3


In [29]:
# Size verhindert diesen Effekt
df_students_nan.groupby('subject').size()

subject
Business    2
Geo         2
Math        3
dtype: int64

## Schlüssel/Index des Groupby-Objekt

In [30]:
# Da wir nach Subject gruppieren, werden die einzigeartigen Werte der Spalte "subject" der neue Index. 
# Den enstehenden Index können wir auch generieren wenn wir uns die einzigartigen Werte der Spalte "subject" anschauen
df_students['subject'].unique()

array(['Math', 'Geo', 'Business'], dtype=object)

## Aggregation von Daten

Wichtigste Funktionen für die Aggregation:
* ``.sum()`` - *Summe*
* ``.mean()`` - *Mittelwert*
* ``.min()`` - *Minimalwert*
* ``.max()`` - *Maximalwert*

In [31]:
# Bilden der Summe nach der "subject_id", text wird ignoriert
df_students.groupby('subject_id').sum()

,student_id,age,left_hand
subject_id,,,
1,309,60,2.0
2,205,48,1.0
3,207,52,0.0


In [32]:
# Durchschnittsalter berechnen
df_students[['subject','age']].groupby('subject').mean()

,age
subject,
Business,26
Geo,24
Math,20


## Auszug aus dem Dataframe betrachten
Mit ``.first()`` und ``.last()`` bekommen wir die einen neuen Dataframe zurück in dem nur der erste oder der letzte Datensatz einer jeden Gruppe enthalten ist. Dies kann besonders nützlich sein, wenn die Datensätze in einer bestimmen Reihenfolge im Datensatz auftauchen.

In [33]:
df_students.groupby('subject').first()

,student_id,subject_id,age,left_hand
subject,,,,
Business,102,3,31,False
Geo,101,2,22,True
Math,100,1,18,False


In [34]:
df_students.groupby('subject').first()

,student_id,subject_id,age,left_hand
subject,,,,
Business,102,3,31,False
Geo,101,2,22,True
Math,100,1,18,False


**Auch ``.head()`` und ``.tail()`` lassen sich hier wie gewohnt verwenden**

In [35]:
df_students.groupby('subject').head(2)

,student_id,subject_id,subject,age,left_hand
0,100,1,Math,18,False
1,101,2,Geo,22,True
2,102,3,Business,31,False
3,103,1,Math,22,True
4,104,2,Geo,26,False
5,105,3,Business,21,False


## Mehrere Gruppierungsebenen
Oft bleibt es nicht bei einer Gruppierungsebene. Wie können beliebig viele Gruppierungsebenen hinzufügen. Die Reihenfolge hat zunächst keinen Erkennbaren Einfluss auf das Endergebnis, außer, dass sich die Sortierung der Daten ändert.

In [36]:
# Größeres Datenset einlesen
df = pd.read_csv('../src/bigdata/120-years-of-olympic-history-athletes-and-results/athlete_events.csv')

In [37]:
# Auswählen von ein paar wahren Champions
df = df[df['ID'].isin([69210, 107383, 127501])]
df

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
137689,69210,"Frederick Carlton ""Carl"" Lewis",M,23.0,188.0,80.0,United States,USA,1984 Summer,1984,Summer,Los Angeles,Athletics,Athletics Men's 100 metres,Gold
137690,69210,"Frederick Carlton ""Carl"" Lewis",M,23.0,188.0,80.0,United States,USA,1984 Summer,1984,Summer,Los Angeles,Athletics,Athletics Men's 200 metres,Gold
137691,69210,"Frederick Carlton ""Carl"" Lewis",M,23.0,188.0,80.0,United States,USA,1984 Summer,1984,Summer,Los Angeles,Athletics,Athletics Men's 4 x 100 metres Relay,Gold
137692,69210,"Frederick Carlton ""Carl"" Lewis",M,23.0,188.0,80.0,United States,USA,1984 Summer,1984,Summer,Los Angeles,Athletics,Athletics Men's Long Jump,Gold
137693,69210,"Frederick Carlton ""Carl"" Lewis",M,27.0,188.0,80.0,United States,USA,1988 Summer,1988,Summer,Seoul,Athletics,Athletics Men's 100 metres,Gold
137694,69210,"Frederick Carlton ""Carl"" Lewis",M,27.0,188.0,80.0,United States,USA,1988 Summer,1988,Summer,Seoul,Athletics,Athletics Men's 200 metres,Silver
137695,69210,"Frederick Carlton ""Carl"" Lewis",M,27.0,188.0,80.0,United States,USA,1988 Summer,1988,Summer,Seoul,Athletics,Athletics Men's Long Jump,Gold
137696,69210,"Frederick Carlton ""Carl"" Lewis",M,31.0,188.0,80.0,United States,USA,1992 Summer,1992,Summer,Barcelona,Athletics,Athletics Men's 4 x 100 metres Relay,Gold
137697,69210,"Frederick Carlton ""Carl"" Lewis",M,31.0,188.0,80.0,United States,USA,1992 Summer,1992,Summer,Barcelona,Athletics,Athletics Men's Long Jump,Gold
137698,69210,"Frederick Carlton ""Carl"" Lewis",M,35.0,188.0,80.0,United States,USA,1996 Summer,1996,Summer,Atlanta,Athletics,Athletics Men's Long Jump,Gold


In [38]:
# Uns interessieren die Medallien und zwar nur die Gewinne
df['Medal'].unique()

array(['Gold', 'Silver', nan, 'Bronze'], dtype=object)

**Umgang mit fehlenden oder fehlerhaften Werten, mit ``.dropna()`` und ``.fillna()``**

In [39]:
# Eine Möglichkeit ist die Daten zu verwerfen
df_drop = df.dropna(subset=['Medal'])
print(len(df))
print(len(df_drop))

34
27


In [40]:
# Bessser jedoch ist, wenn wir den fehlenden Daten eine Bedeutung und einen Wert zukommen lassen
df['Medal'] = df['Medal'].fillna('Teilnahme')
df.sample(10)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
254687,127501,Mikhail Yakovlevich Voronin,M,23.0,170.0,65.0,Soviet Union,URS,1968 Summer,1968,Summer,Mexico City,Gymnastics,Gymnastics Men's Floor Exercise,Teilnahme
213830,107383,"Donald Arthur ""Don"" Schollander",M,22.0,180.0,79.0,United States,USA,1968 Summer,1968,Summer,Mexico City,Swimming,Swimming Men's 4 x 200 metres Freestyle Relay,Gold
254693,127501,Mikhail Yakovlevich Voronin,M,27.0,170.0,65.0,Soviet Union,URS,1972 Summer,1972,Summer,Munich,Gymnastics,Gymnastics Men's Individual All-Around,Teilnahme
254692,127501,Mikhail Yakovlevich Voronin,M,23.0,170.0,65.0,Soviet Union,URS,1968 Summer,1968,Summer,Mexico City,Gymnastics,Gymnastics Men's Pommelled Horse,Bronze
254694,127501,Mikhail Yakovlevich Voronin,M,27.0,170.0,65.0,Soviet Union,URS,1972 Summer,1972,Summer,Munich,Gymnastics,Gymnastics Men's Team All-Around,Silver
213831,107383,"Donald Arthur ""Don"" Schollander",M,22.0,180.0,79.0,United States,USA,1968 Summer,1968,Summer,Mexico City,Swimming,Swimming Men's 4 x 100 metres Medley Relay,Gold
254690,127501,Mikhail Yakovlevich Voronin,M,23.0,170.0,65.0,Soviet Union,URS,1968 Summer,1968,Summer,Mexico City,Gymnastics,Gymnastics Men's Horizontal Bar,Gold
137698,69210,"Frederick Carlton ""Carl"" Lewis",M,35.0,188.0,80.0,United States,USA,1996 Summer,1996,Summer,Atlanta,Athletics,Athletics Men's Long Jump,Gold
254696,127501,Mikhail Yakovlevich Voronin,M,27.0,170.0,65.0,Soviet Union,URS,1972 Summer,1972,Summer,Munich,Gymnastics,Gymnastics Men's Horse Vault,Teilnahme
137695,69210,"Frederick Carlton ""Carl"" Lewis",M,27.0,188.0,80.0,United States,USA,1988 Summer,1988,Summer,Seoul,Athletics,Athletics Men's Long Jump,Gold


In [41]:
# Teilnehmer ist einzigartig zugeordnet über die ID. Sie bildet das oberste Level.
df.groupby(['ID', 'Name', 'Team', 'Medal']).size()

ID      Name                             Team           Medal    
69210   Frederick Carlton "Carl" Lewis   United States  Gold         9
                                                        Silver       1
107383  Donald Arthur "Don" Schollander  United States  Gold         7
                                                        Silver       1
127501  Mikhail Yakovlevich Voronin      Soviet Union   Bronze       1
                                                        Gold         2
                                                        Silver       6
                                                        Teilnahme    7
dtype: int64

In [42]:
df.groupby(['Medal', 'ID', 'Name', 'Team']).size()

Medal      ID      Name                             Team         
Bronze     127501  Mikhail Yakovlevich Voronin      Soviet Union     1
Gold       69210   Frederick Carlton "Carl" Lewis   United States    9
           107383  Donald Arthur "Don" Schollander  United States    7
           127501  Mikhail Yakovlevich Voronin      Soviet Union     2
Silver     69210   Frederick Carlton "Carl" Lewis   United States    1
           107383  Donald Arthur "Don" Schollander  United States    1
           127501  Mikhail Yakovlevich Voronin      Soviet Union     6
Teilnahme  127501  Mikhail Yakovlevich Voronin      Soviet Union     7
dtype: int64

**Gruppierung zurücksetzen und in "normalen" DataFrame zurückverwandeln mit ``.reset_index()`` und umbennen von Spalten mit ``.rename()``**  

In [45]:
# Zurück wandeln eines "grouped" DataFrames mit .reset_index() 
df_norm = df.groupby(['ID', 'Name', 'Team', 'Medal']).size().reset_index()
df_norm = df_norm.rename(columns={0:'Anzahl'})
df_norm.sort_values(['Name', 'Anzahl'], ascending=[True, False])

,ID,Name,Team,Medal,Anzahl
2,107383,"Donald Arthur ""Don"" Schollander",United States,Gold,7
3,107383,"Donald Arthur ""Don"" Schollander",United States,Silver,1
0,69210,"Frederick Carlton ""Carl"" Lewis",United States,Gold,9
1,69210,"Frederick Carlton ""Carl"" Lewis",United States,Silver,1
7,127501,Mikhail Yakovlevich Voronin,Soviet Union,Teilnahme,7
6,127501,Mikhail Yakovlevich Voronin,Soviet Union,Silver,6
5,127501,Mikhail Yakovlevich Voronin,Soviet Union,Gold,2
4,127501,Mikhail Yakovlevich Voronin,Soviet Union,Bronze,1


# Tipps and Tricks

###  Referenz aller Funktionen des groupby-Objekt
* https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html#computations-descriptive-stats

### Vollständige Tutorial zu Gruppieren aus der offiziellen Pandas Dokumentation
* https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html
* https://pandas.pydata.org/pandas-docs/stable/user_guide/cookbook.html#cookbook-grouping
    
